In [9]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.6 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
p = '/content/gdrive/MyDrive/audio'

In [4]:
for i in os.listdir(p):
    print(os.path.join(p,i))

/content/gdrive/MyDrive/audio/wav1.wav
/content/gdrive/MyDrive/audio/wav2.wav
/content/gdrive/MyDrive/audio/wav3.wav
/content/gdrive/MyDrive/audio/wav4.wav
/content/gdrive/MyDrive/audio/wav5.wav
/content/gdrive/MyDrive/audio/wav1_4.wav
/content/gdrive/MyDrive/audio/wav1_2.wav
/content/gdrive/MyDrive/audio/wav1_3.wav
/content/gdrive/MyDrive/audio/wav1_5.wav
/content/gdrive/MyDrive/audio/wav1_1.wav


In [17]:
import librosa
import numpy as np

arr = []
arr_2 = []
for i in sorted(os.listdir(p)):
# Load the audio file
    print(i)
    audio, sr = librosa.load(os.path.join(p,i), sr=16000)

    # Split the audio into segments of 1 second
    segments = librosa.effects.split(audio, top_db=30, frame_length=sr, hop_length=sr)

    # Create an empty list to store the spectrograms
    spectrograms = []

    # Iterate through the segments
    for segment in segments:
        
        # Apply a Hann window to the segment
        audio_segment = audio[segment[0]:segment[1]]
        
        # Apply Hann window to segment
        window = np.hanning(len(audio_segment))
        audio_segment = audio_segment * window
        arr_2.append(audio_segment)
        # Compute the STFT of the segment
        stft = librosa.core.stft(audio_segment, n_fft=2048, hop_length=512)
        
        # Compute the magnitude spectrogram
        spectrogram = np.abs(stft)
        print(spectrogram.shape)
        # Normalize the spectrogram by subtracting the mean and dividing by the standard deviation
        mean = np.mean(spectrogram)
        std = np.std(spectrogram)
        spectrogram = (spectrogram - mean) / std
        
        # Append the spectrogram to the list
        spectrograms.append(spectrogram)

    # Convert the list of spectrograms to a numpy array
    spectrograms = np.array(spectrograms)
    arr.append(spectrograms)
    # Use the spectrograms as input for fine-tuning the Wav2Vec2-base model


wav1.wav
(1025, 160)
wav1_1.wav
(1025, 157)
wav1_2.wav
(1025, 109)
wav1_3.wav
(1025, 177)
wav1_4.wav
(1025, 189)
wav1_5.wav
(1025, 139)
wav2.wav
(1025, 185)
wav3.wav
(1025, 248)
wav4.wav
(1025, 319)
wav5.wav
(1025, 271)


In [18]:
len(arr_2)

10

In [27]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("facebook/wav2vec2-base")

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['project_hid.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_q.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
arr[0].shape

(1, 1025, 160)

In [ ]:
import torch
import torch.nn.functional as F
spectrogram_1 = torch.from_numpy(arr_2[1])
# spectrogram_1 = spectrogram_1.squeeze(0)
# spectrogram_1 = spectrogram_1.float()
expected_size = (1025, 512)
current_size = spectrogram_1.shape
# padding = (expected_size[0] - current_size[0], expected_size[1] - current_size[1])

# Pad the spectrogram
# spectrogram_padded = F.pad(spectrogram_1, padding, mode='constant', value=0)

# Pass the padded spectrogram through the model
output = model(spectrogram_1)


In [28]:
output['last_hidden_state'].squeeze(1).shape

torch.Size([1025, 768])

In [ ]:
import pandas as pd
import torch
import torch.nn.functional as F
# create a list to store the outputs
output_list = []

# loop through the dataset and get the output from the model
for i in range(1):
    spectrogram_1 = torch.from_numpy(arr[i])
    spectrogram_1 = spectrogram_1.squeeze(0)
    spectrogram_1 = spectrogram_1.float()
    expected_size = (1025, 512)
    current_size = spectrogram_1.shape
    padding = (expected_size[0] - current_size[0], expected_size[1] - current_size[1])

    # Pad the spectrogram
    spectrogram_padded = F.pad(spectrogram_1, padding, mode='constant', value=0)

    # Pass the padded spectrogram through the model
    output = model(spectrogram_padded)
    output_list.append(output['last_hidden_state'].squeeze(1))

# convert the output list to a dataframe


In [34]:
df = pd.DataFrame(output_list[0].detach().numpy())

# save the dataframe to a csv file
df.to_csv('outputs.csv', index=False)


In [36]:
import torch
from torch.utils.data import Dataset, DataLoader

class AudioDataset(Dataset):
    def __init__(self, audio_files, labels):
        self.audio_files = audio_files
        self.labels = labels

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        audio = self.audio_files[idx]
        label = self.labels[idx]
        return audio, label


In [37]:
audio_files = arr
labels = [1,0,0,0,0,0,1,1,1,1]
dataset = AudioDataset(audio_files, labels)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define your classifier
class Classifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        out = self.fc(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# Train the classifier on your dataset
for epoch in range(5):
    for i, (audio_logits, labels) in enumerate(dataloader):
        # Convert the logits and labels to Pytorch tensors
        audio_logits = torch.from_numpy(audio_logits)
        labels = torch.from_numpy(labels)

        # Forward pass
        outputs = Classifier(audio_logits)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
